# Building lammps simulation 

This notebooks automatically builds the initial systems to be simulated with lammps. 
For instructions, etc, see the rest of this document. As mentioned in `README.md` the code uses a different set of mathematical symbols wrt the article. Specifically, the symbols used in our article become:
- $n$ -> $M$
- $m$ -> $n$
- $n_{tw}$ -> $M_{alt}$
- Furthermore, the parameters $L_k$ below goes from $0$ when $n_{tw}=0$ to $1$ when $n_{tw}=n/2$

### Catenane building algorithm
The building algorithm is implemented in `ror.builder` and works as follows
1. Decide the total number of distorted rings to be placed (dodecagons), $M_{distorted}$
2. Decide the number of dodecagons linking with +2 link (alternating), $M_{alt}$
3. Build a n-gon with $M_{distorted}$ circles placed on its vertices
4. Build a n-gon with $M_{distorted}$ sides, the first $M_{alt}$ rings being alternating distorted and the following contributin 0 to the total linking number
5. Rotate the first n-gon, the one with the circles, by an angle $\frac{\alpha}{2} = \frac{\pi}{M_{distorted}}$, $M_{distorted}$ coincides with the number of sides of the n-gon
6. Collate the coordinates of the circles and distorted rings to form the final catenane 

### Assumption
1. The beads have diameter $\sigma=1$ (LJ units)
2. The total number of rings is always even and equal to $M = 2 M_{distorted}$

### Notes
The setup is a bit crude, but flexible and allows the user to copy this file and creates different sets of catenanes.

The sets of catenanes built by this system are an example, and do not cover the whole set of parameters used in the main text.

Ideally, one should have a single class to write a lammps simulation in a flexible way. This is basically a project in itself, so I will not follow it here. 

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src/'))
import plotly.express as px
from pathlib import Path
from jinja2 import Environment, FileSystemLoader

In [2]:
from ror.builder import Catenane, CatenaneBuilder
from ror.builder import LammpsLangevinInput, LammpsDatafile

### Folders to be used

In [3]:
data_dir = Path('../data/')
output_dir = data_dir / '01-lammps-init'
template_dir = '../src/ror/templates/'

## Systems to build

The systems are built in two steps. The first one serve to specify the beads per elementary rings and the number of distorted rings $M_{distorted}$, to obtain a catenane with $M=2M_{distorted}$ rings. 

The second step, done below, serves to specify the fraction of distorted rings that do introduce a twist. This is called $Lk$ in the file names. 

The scheme below produces the starting configurations and lammps file for all catenenas used in our study:
$n_{tw}=0, 0.5n$, $20$ rings and $n=144$, $n=36$ beads, and 5 catenanes with $n_{tw}\in \{0, 0.1,0.2,0.3,0.4,0.5\} n $ for $m=48$ and $n=20,40,60,80,100,200,300$.

In [33]:
builders_n48 = (CatenaneBuilder(48,M) for M in (10,20,30,40,50,100,150))
builders_M20n144n36 = (CatenaneBuilder(36,10), CatenaneBuilder(144,10))

For each catenane  with $M=20$ we consider for now two values of $Lk$, $Lk \in $ {0, 1}. 
- $1.0$ means that all links are $+1$, so that $n_{tw}=n/2$.
- $0.0 $ means that the links alternate between $-1$ and $+1$, so that their sum is 0 and $n_{tw}=0$ 

Fractional values between the two are possible and correspond to place a certain fraction of $+1,+1$ vs $+1,-1$ rings on the catenane. Seen another way, $Lk$ indicates the fraction of odd rings that induce a twist. 


In [35]:
systems = {}
for cat in builders_M20n144n36:
    for lk in (0.0, 1.0):
        systems[f'cat_M{cat.M}n{cat.n}Lk{lk:.3f}'] = cat.build_system(lk) 

for cat in builders_n48:
    for lk in (0.0, 0.2, 0.4, 0.6, 0.8, 1.0):
        systems[f'cat_M{cat.M}n{cat.n}Lk{lk:.3f}'] = cat.build_system(lk) 



The keys of the dictionary correspond to the names of the datafiles to be used for simulation and analysis

In [36]:
print(systems.keys())

dict_keys(['cat_M20n36Lk0.000', 'cat_M20n36Lk1.000', 'cat_M20n144Lk0.000', 'cat_M20n144Lk1.000', 'cat_M20n48Lk0.000', 'cat_M20n48Lk0.200', 'cat_M20n48Lk0.400', 'cat_M20n48Lk0.600', 'cat_M20n48Lk0.800', 'cat_M20n48Lk1.000', 'cat_M40n48Lk0.000', 'cat_M40n48Lk0.200', 'cat_M40n48Lk0.400', 'cat_M40n48Lk0.600', 'cat_M40n48Lk0.800', 'cat_M40n48Lk1.000', 'cat_M60n48Lk0.000', 'cat_M60n48Lk0.200', 'cat_M60n48Lk0.400', 'cat_M60n48Lk0.600', 'cat_M60n48Lk0.800', 'cat_M60n48Lk1.000', 'cat_M80n48Lk0.000', 'cat_M80n48Lk0.200', 'cat_M80n48Lk0.400', 'cat_M80n48Lk0.600', 'cat_M80n48Lk0.800', 'cat_M80n48Lk1.000', 'cat_M100n48Lk0.000', 'cat_M100n48Lk0.200', 'cat_M100n48Lk0.400', 'cat_M100n48Lk0.600', 'cat_M100n48Lk0.800', 'cat_M100n48Lk1.000', 'cat_M200n48Lk0.000', 'cat_M200n48Lk0.200', 'cat_M200n48Lk0.400', 'cat_M200n48Lk0.600', 'cat_M200n48Lk0.800', 'cat_M200n48Lk1.000', 'cat_M300n48Lk0.000', 'cat_M300n48Lk0.200', 'cat_M300n48Lk0.400', 'cat_M300n48Lk0.600', 'cat_M300n48Lk0.800', 'cat_M300n48Lk1.000'])


### Visualizing the systems
This can be done using plotly. In the figures below, "alternate" means "twist_inducing" (+2 or -2 ring depending on how the normals are picked), "circle" a planar circle and "same" a distorted ring which does not induce twist (0 ring). 

In [23]:
px.scatter_3d(systems['cat_M20n48Lk0.400'].data,x='x',y='y',z='z',color='type',range_z=[-20,20])

In [24]:
px.scatter_3d(systems['cat_M100n48Lk0.800'].data,x='x',y='y',z='z',color='type',range_z=[-20,20])

## Building  the simulations starting files
With the data generated above we are now ready to build the lammps init files. To do so, we need the simulation parameters below. To each simulation corresponds a different directory in `data/01-lammps-init`. These directory names will further be used for analysis etc.

Each folder contains:
- a lammps input file
- a lammps data file
- a pbs file to launch the simulation on SGE managed clusters
- a slurm filt to launch the simulation on SLURM managed clusters


**The bending of the catenane rings has not been fixed yet, we set it to $k_b = 2*n$ with $n$ the number of beads in each ring.**

In [37]:
simulation_params = {'steps':1e9, 'n_thermo':1e4, 'n_dump':1e6, 'n_balance':5e5, 'tau_damp':10}

In [38]:
file_loader = FileSystemLoader(LammpsLangevinInput.template_dir)
env = Environment(loader=file_loader)
name_pbs = 'pbs_template.pbs'
name_slurm = 'slurm_template.slrm'

template_pbs = env.get_template(name_pbs)
template_slurm = env.get_template(name_slurm)

## Batch scheduler parameters
Change the parameters according to the batch scheduler that is used.

In [39]:
hpcparams_pbs = {'queue':'short_cpuQ',
                 'hours':5,'minutes':55}
hpcparams_slurm = {'hours':72,'minutes':0}

In [40]:
def set_cpus(total_beads):
    n = total_beads
    if n < 1000:
        return 4
    elif n < 3000:
        return 4
    elif n < 5000:
        return 6
    else:
        return 16

Creates the starting directory populated with all needed files in `../data/01-lammps-init`

In [41]:
for name, data in systems.items():
    output_locdir = output_dir / name
    os.mkdir (output_locdir)
    lmp_data = LammpsDatafile(data)
    lmp_data.to_file(output_locdir /  (name + '.dat'))
    lmp_input = LammpsLangevinInput (2*data.n, **simulation_params)
    lmp_input.to_file( name +'.dat', output_locdir / (name + '.lmp'))
    with open(output_locdir / (name + '.slrm'),'w') as fslrm:
        hpcparams_slurm['input'] = name + '.lmp'
        hpcparams_slurm['name'] = name
        fslrm.write(template_slurm.render(**hpcparams_slurm))
    with open(output_locdir / (name + '.pbs'),'w') as fpbs:
        hpcparams_pbs['name'] = name
        hpcparams_pbs['input'] = name + '.lmp'
        n_cpus = set_cpus(data.n * data.M)
        hpcparams_pbs['n_cpus'] = n_cpus
        hpcparams_pbs['n_mpi'] = n_cpus
        fpbs.write(template_pbs.render(**hpcparams_pbs))